# Exercise 4
Write a discrete event simulation program for a blocking system,
i.e. a system with m service units and no waiting room. The offered
traffic A is the product of the mean arrival rate and the mean
service time
## 1
The arrival process is modelled as a Poisson process. Report the
fraction of blocked customers, and a confidence interval for this
fraction. Choose the service time distribution as exponential.
Parameters: m = 10, mean service time = 8 time units, mean
time between customers = 1 time unit (corresponding to an
offered traffic of 8 Erlang), 10 x 10.000 customers.


In [1]:
import numpy as np
#import poission
import math
from scipy.stats import poisson
#import exponential
from scipy.stats import expon
import bisect

In [2]:
m = 10 #number of servers
s = 8 #mean service time
lam = 1#arrival_intensity
total_customers =10000 #10*10000
A = lam*s

In [3]:
class Customer:
    def __init__(self, arrival_time, service_time):
        self.service_time = service_time
        self.blocked = False
        
        self.event = "arrival"
        self.event_time = arrival_time
      
                
    def arrive(self, servers, event_list):
        if servers < 1:
            self.blocked = True
            return servers
        else:
            servers -= 1
            servers = max(servers, 0)
            self.event = "departure"
            self.event_time += self.service_time
            bisect.insort(event_list, self, key=lambda x: x.event_time)
            return servers
    
    def depart(self, servers):
        servers += 1
        servers = min(servers, m)
        return servers

In [4]:
#arrival time differences are exponentially distributed
arrival_intervals = np.random.exponential(1/lam, total_customers)
arrival_times = np.cumsum(arrival_intervals)

In [5]:

event_list = [Customer(arrival_time, expon.rvs(scale = s)) for arrival_time in arrival_times]
def main_loop(event_list, m, repititions = 10):
    blocked = np.zeros(repititions)
    for i in range(repititions):
        event_list.sort(key=lambda x: x.event_time)
        open_servers = m
        while event_list:
            event = event_list.pop(0)
            if event.event == "arrival":
                open_servers = event.arrive(open_servers, event_list)
                blocked[i] += event.blocked
            elif event.event == "departure":
                open_servers = event.depart(open_servers)
        return blocked

blocked_1 = main_loop(event_list, m)

In [6]:
print("Blocking probability: ", blocked_1/total_customers)

Blocking probability:  [0.1226 0.     0.     0.     0.     0.     0.     0.     0.     0.    ]


In [7]:
#Erlang B formula
def erlang_b(m, A):
    return (A**m/math.factorial(m))/np.sum([A**i/math.factorial(i) for i in range(m+1)])

In [8]:
#Theoretical blocking probability
print(erlang_b(m, A))

0.12166106425295149


## 2
The arrival process is modelled as a renewal process using the
same parameters as in Part 1 when possible. Report the
fraction of blocked customers, and a confidence interval for this
fraction for at least the following two cases

In [9]:
# (a) Experiment with Erlang distributed inter arrival times The
#Erlang distribution should have a mean of 1
inter_arrival_times = np.random.gamma(m, 1/m, total_customers)
arrabival_times = np.cumsum(inter_arrival_times)
event_list = [Customer(arrival_time, expon.rvs(scale = s)) for arrival_time in arrival_times]
blocked_erlang = main_loop(event_list, m)
print("Blocking probability: ", blocked_erlang/total_customers)

Blocking probability:  [0.1196 0.     0.     0.     0.     0.     0.     0.     0.     0.    ]


In [10]:
# hyper exponential inter arrival times. The parameters for
#the hyper exponential distribution should be
p1 = 0.8
λ1 = 0.8333
p2 = 0.2
λ2 = 5.0
inter_arrival_times = np.random.choice([expon.rvs(scale = 1/λ1), expon.rvs(scale = 1/λ2)], total_customers, p=[p1, p2])
arrival_times = np.cumsum(inter_arrival_times)
event_list = [Customer(arrival_time, expon.rvs(scale = s)) for arrival_time in arrival_times]
blocked_hyperexp = main_loop(event_list, m)
print("Blocking probability: ", blocked_hyperexp/total_customers)



Blocking probability:  [0.9869 0.     0.     0.     0.     0.     0.     0.     0.     0.    ]


## 3
The arrival process is again a Poisson process like in Part 1.
Experiment with different service time distributions with the
same mean service time and m as in Part 1 and Part 2

In [11]:
# a) Constant service time
arrival_intervals = np.random.exponential(1/lam, total_customers)
arrival_times = np.cumsum(arrival_intervals)
event_list = [Customer(arrival_time, s) for arrival_time in arrival_times]
blocked_constant = main_loop(event_list, m)
print("Blocking probability: ", blocked_constant/total_customers)

Blocking probability:  [0.1173 0.     0.     0.     0.     0.     0.     0.     0.     0.    ]


In [12]:
# Pareto distributed service times with at least k = 1.05 and
#k = 2.05.

k = 1.05 
event_list = [Customer(arrival_time, np.random.pareto(k)) for arrival_time in arrival_times]
blocked_pareto_1 = main_loop(event_list, m)
print("Blocking probability for k= 1.05: ", blocked_pareto_1/total_customers)
k = 2.05
event_list = [Customer(arrival_time, np.random.pareto(k)) for arrival_time in arrival_times]
blocked_pareto_2 = main_loop(event_list, m)
print("Blocking probability for k= 2.05: ", blocked_pareto_2/total_customers)

Blocking probability for k= 1.05:  [0.0983 0.     0.     0.     0.     0.     0.     0.     0.     0.    ]
Blocking probability for k= 2.05:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [13]:
#absolute gaussian distributed service times with mean s and standard deviation s/4 #99%+ of the values are positive
event_list = [Customer(arrival_time, np.random.normal(s, s/4)) for arrival_time in arrival_times]
blocked = main_loop(event_list, m)
print("Blocking probability: ", blocked/total_customers)

Blocking probability:  [0.1177 0.     0.     0.     0.     0.     0.     0.     0.     0.    ]


In [28]:
def getConfidenceInts(m):
    # Service times
    # Erlang
    a = []
    for i in range(m):
        inter_arrival_times = np.random.gamma(m, 1/m, total_customers)
        arrival_times = np.cumsum(inter_arrival_times)
        event_list = [Customer(arrival_time, expon.rvs(scale = s)) for arrival_time in arrival_times]
        a.append(main_loop(event_list, m, repititions= 1)[0])

    # Hyperexponential
    b = []
    p1 = 0.8
    λ1 = 0.8333
    p2 = 0.2
    λ2 = 5.0
    for i in range(m):
        inter_arrival_times = np.random.choice([expon.rvs(scale = 1/λ1), expon.rvs(scale = 1/λ2)], total_customers, p=[p1, p2])
        arrival_times = np.cumsum(inter_arrival_times)
        event_list = [Customer(arrival_time, expon.rvs(scale = s)) for arrival_time in arrival_times]
        b.append(main_loop(event_list, m, 1)[0])

    # Constant service time
    c = []
    for i in range(m):
        arrival_intervals = np.random.exponential(1/lam, total_customers)
        arrival_times = np.cumsum(arrival_intervals)
        event_list = [Customer(arrival_time, s) for arrival_time in arrival_times]
        c.append(main_loop(event_list, m, 1))

    # Two Paretos
    d = []
    e = []
    k1 = 1.05
    k2 = 2.05
    for i in range(m):
        arrival_intervals = np.random.exponential(1/lam, total_customers)
        arrival_times = np.cumsum(arrival_intervals)
        event_list = [Customer(arrival_time, np.random.pareto(k1)) for arrival_time in arrival_times]
        d.append(main_loop(event_list, m, 1)[0])
       
        event_list = [Customer(arrival_time, np.random.pareto(k2)) for arrival_time in arrival_times]
        e.append( main_loop(event_list, m, 1)[0])
        
    # Absolute normal
    f = []
    for i in range(m):
        event_list = [Customer(arrival_time, np.random.normal(s, s/4)) for arrival_time in arrival_times]
        f.append(main_loop(event_list, m,1)[0])

    # Both exponential
    h = []
    for i in range(m):
        arrival_intervals = np.random.exponential(1/lam, total_customers)
        arrival_times = np.cumsum(arrival_intervals)
        event_list = [Customer(arrival_time, expon.rvs(scale = s)) for arrival_time in arrival_times]
        h.append(main_loop(event_list, m, 1)[0])


    return a, b, c, d, e, f, h

In [29]:
a, b, c, d, e, f, h = getConfidenceInts(40)

In [33]:
print(f"Both exponential:\n Mean: {np.mean(h):.0f}  Standard Deviation: {np.sqrt(np.var(h)):.0f}.\n Confidence interval: [{np.mean(h) + np.sqrt(np.var(h)) / np.sqrt(len(h)) * -1.96:.0f},{np.mean(h) + np.sqrt(np.var(h)) / np.sqrt(len(h)) * 1.96:.0f} ]")

print(f"Arrival Erlang:\n Mean: {np.mean(a):.0f}  Standard Deviation: {np.sqrt(np.var(a)):.0f}.\n Confidence interval: [{np.mean(a) + np.sqrt(np.var(a)) / np.sqrt(len(a)) * -1.96:.0f},{np.mean(a) + np.sqrt(np.var(a)) / np.sqrt(len(a)) * 1.96:.0f} ]")

print(f"Arrival is hyperexponential:\n Mean: {np.mean(b):.0f}  Standard Deviation: {np.sqrt(np.var(b)):.0f}.\n Confidence interval: [{np.mean(b) + np.sqrt(np.var(b)) / np.sqrt(len(b)) * -1.96:.0f},{np.mean(b) + np.sqrt(np.var(b)) / np.sqrt(len(b)) * 1.96:.0f} ]")

print(f"Service is constant :\n Mean: {np.mean(c):.0f}  Standard Deviation: {np.sqrt(np.var(c)):.0f}.\n Confidence interval: [{np.mean(c) + np.sqrt(np.var(c)) / np.sqrt(len(c)) * -1.96:.0f},{np.mean(c) + np.sqrt(np.var(c)) / np.sqrt(len(c)) * 1.96:.0f} ]")

print(f"Service is Pareto k = 1.05:\n Mean: {np.mean(d):.0f}  Standard Deviation: {np.sqrt(np.var(d)):.0f}.\n Confidence interval: [{np.mean(d) + np.sqrt(np.var(d)) / np.sqrt(len(d)) * -1.96:.0f},{np.mean(d) + np.sqrt(np.var(d)) / np.sqrt(len(d)) * 1.96:.0f} ]")

print(f"Service is Pareto k = 2.05:\n Mean: {np.mean(e):.0f}  Standard Deviation: {np.sqrt(np.var(e)):.0f}.\n Confidence interval: [{np.mean(e) + np.sqrt(np.var(e)) / np.sqrt(len(e)) * -1.96:.0f},{np.mean(e) + np.sqrt(np.var(e)) / np.sqrt(len(e)) * 1.96:.0f} ]")

print(f"Service is Absolute Normal\n Mean: {np.mean(f):.0f}  Standard Deviation: {np.sqrt(np.var(f)):.0f}.\n Confidence interval: [{np.mean(f) + np.sqrt(np.var(f)) / np.sqrt(len(f)) * -1.96:.0f},{np.mean(f) + np.sqrt(np.var(f)) / np.sqrt(len(f)) * 1.96:.0f} ]")

Both exponential:
 Mean: 1097  Standard Deviation: 116.
 Confidence interval: [1061,1134 ]
Arrival Erlang:
 Mean: 313  Standard Deviation: 118.
 Confidence interval: [276,349 ]
Arrival is hyperexponential:
 Mean: 2322  Standard Deviation: 2601.
 Confidence interval: [1516,3128 ]
Service is constant :
 Mean: 1069  Standard Deviation: 122.
 Confidence interval: [1031,1107 ]
Service is Pareto k = 1.05:
 Mean: 602  Standard Deviation: 340.
 Confidence interval: [496,707 ]
Service is Pareto k = 2.05:
 Mean: 0  Standard Deviation: 0.
 Confidence interval: [0,0 ]
Service is Absolute Normal
 Mean: 1035  Standard Deviation: 107.
 Confidence interval: [1002,1068 ]
